
# Swap Test Tutorial

## Introduction

The swap test is a quantum computing technique used to determine the similarity between two quantum states. It plays a crucial role in quantum information processing, enabling the measurement of the overlap between states without directly observing their contents. This tutorial covers the importance, utility, applications, and operation of the swap test in quantum computing. This tutorial also showcases the powerful abstractions provide by the Classiq platform and SDK, highlighting how Classiq makes quantum algorithm design, development, prototyping, and testing streamlined. 

## Why It's Important

Quantum computing operates on the principles of quantum mechanics, leveraging superposition and entanglement to perform computations that are infeasible for classical computers. The swap test is important for several reasons:

- **State Comparison**: It provides a means to compare quantum states without collapsing their superpositions, preserving the quantum information.
- **Entanglement Detection**: Helps in detecting entanglement, a key resource in quantum computing and communication.
- **Quantum Algorithms**: Integral to the implementation of various quantum algorithms, enhancing their efficiency and applicability.

## How It's Useful

The swap test is useful in a variety of quantum computing tasks:

- **Quantum Machine Learning**: In algorithms that require measuring the distance or similarity between vectors (states), such as clustering and classification.
- **Quantum Chemistry**: For comparing molecular states in simulations, aiding in the understanding of chemical reactions at the quantum level.
- **Quantum Cryptography**: In protocols where verifying the integrity of quantum states is necessary, ensuring secure communication.

## Where It's Useful

Applications of the swap test span across multiple domains, demonstrating its versatility:

- **Algorithmic Benchmarking**: Testing and comparing the performance of quantum algorithms in solving specific problems.
- **Quantum Error Correction**: Assessing the fidelity of quantum error correction codes by comparing corrected states against ideal states.
- **Entanglement Verification**: In experimental setups to verify the presence and degree of entanglement between quantum systems.

## Swap Test

The swap test algorithm operates by creating an interference pattern to measure the overlap between two quantum states. Here's a simplified explanation of its operation:

1. **Preparation**: Two quantum states $|\psi\rangle$ and $|\phi\rangle$ are prepared, whose similarity is to be tested.

2. **Ancilla Qubit**: An additional qubit (ancilla) is initialized in the state $|0\rangle$.

3. **Hadamard Operation**: A Hadamard gate is applied to the ancilla qubit, putting it into a superposition state of $\frac{|0\rangle + |1\rangle}{\sqrt{2}}$.

4. **Controlled Swap**: A controlled swap (Fredkin gate) operation is performed between the two quantum states, conditioned on the state of the ancilla qubit.

5. **Second Hadamard**: Another Hadamard gate is applied to the ancilla qubit.

6. **Measurement**: The ancilla qubit is measured. The probability of finding it in the state $|0\rangle$ is related to the overlap (similarity) between the states $|\psi\rangle$ and $|\phi\rangle$.<br><br>

---

## Getting Started

- Authenticate the Classiq platform with your device. 
- Import the classiq SDK and necessary member functions



In [ ]:
import classiq
classiq.authenticate()

In [ ]:
from classiq import *


---

## Example One: Comparing Random Quantum States

### Objective
Compare two quantum states, $|\psi\rangle$ and $|\phi\rangle$, using the `swap_test()` method from the Classiq SDK.

### Process Overview

#### Step 1: Generation of Quantum States

- **Random State Creation**: Generate random amplitudes for two quantum states.
- **Quantum State Preparation**: Utilize `prepare_amplitudes` from the Classiq SDK to initialize quantum states of qubits based on these amplitudes.

### Implementation Details

Generate two arrays of random numbers within the range [-1, 1]. The array size equals the number of possible superposition states for `NUM_QUBITS`, aligning with the quantum system's complexity.

In [ ]:
import numpy as np

np.random.seed(18)

NUM_QUBITS = 4
amps1 = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amps2 = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amps1 = amps1 / np.linalg.norm(amps1)
amps2 = amps2 / np.linalg.norm(amps2)


### Step Two: Use the Random States to Prepare $|\psi\rangle$ and $|\phi\rangle$
- First we instantiate two QArrays that will be $|\psi\rangle$ and $|\phi\rangle$. Next we uses the randomly generated arrays `amps1` and `amps2` to `prepare_amplitudes` for `state1` and `state2`. 

### Step Three: Swap Test
- Now that $|\psi\rangle$ and $|\phi\rangle$ have been prepared, we may preform a `swap_test` to compare the quantum states. 

With a simple function call `swap_test` handles the functionality for us! Notice that ancilla is not allocated any space previous to `swap_test`, this allows for the function to handle various input sizes. Preparing amplitudes and using Swap Test are two small examples of how the Classiq platform and SDK provide many high-level abstractions that make quantum algorithm development, prototyping, and testing more efficient!!!



In [ ]:
@QFunc
def main(ancilla:Output[QBit]):
    state1 = QArray("state1")
    state2 = QArray("state2")
    prepare_amplitudes(amps1.tolist(), 0.0, state1)
    prepare_amplitudes(amps2.tolist(), 0.0, state2)
    swap_test(state1, state2, ancilla)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Lets look at the results of our quantum program. Execute the program from the SDK below. We will compare the Swap-Test result with an exact classical calculation to test accuracy of our result.

In [ ]:
result = execute(qprog).result()

state_overlap = np.sqrt(
    2 * result[0].value.counts["0"] / sum(result[0].value.counts.values()) - 1
)

exact_overlap = np.abs(amps1 @ amps2)
print(f"States overlap from Swap-Test result: {state_overlap}")
print(f"States overlap from classical calculation: {exact_overlap}")

assert np.isclose(state_overlap, exact_overlap, atol=1e-2), "Overlap is not accurate enough"

### Key Points

- **Random Amplitudes**: Represent the quantum states $|\psi\rangle$ and $|\phi\rangle$.
- **Normalization**: Essential for quantum state validity, ensuring the probability sum equals 1.
- **Classiq SDK's Role**: Facilitates quantum state preparation and comparison through specific functions like `prepare_amplitudes` and `swap_test()`. 

This streamlined approach emphasizes the key steps and functions involved in generating and comparing quantum states, providing a clear and concise understanding of the process.

This has been a simple yet intuitive example of Swap Test and how it can be used to compare two quantum states, $|\psi\rangle$ and $|\phi\rangle$. Lets take a look at a more complex example to further explore Swap Test and the power of the Classiq Platform and SDK.<br><br><br>

---

## Example Two: Quadratic Function Comparison via Swap Test

Explore the behavior of two quadratic functions by employing various coefficients and input values. Utilize the Swap Test to analyze and compare the outcomes.

### Step One: Implementing an In-Place Quadratic Function
- **Objective**: Develop a quantum function capable of computing $y = a \cdot x^2$ in-place, where $a$ acts as a variable coefficient. This function should directly encode the result of the quadratic equation into a quantum state, leveraging the unique computational capabilities of quantum systems for processing and analysis.

In [ ]:
@QFunc
def inplace_quadratic(a:QParam[int], x:QNum, y: Output[QNum]):
    y |= a*x**2

WOW, who would've expected implementing a quadratic function in a quantum program could be this straightforward? Thanks Classiq!

### Step Two: Assembly and Comparison

- **Objective**: Craft two quadratic expressions for examination. In this scenario, the quadratics will be identical for the sake of comparison.
- **Superposition of Inputs**: Initialize the input \(x\) into a quantum state of superposition, ensuring its range spans $[0,2^2)$. This step is crucial for exploiting quantum parallelism, allowing the simultaneous evaluation of the quadratic function at multiple points.
- **Comparative Analysis with Swap Test**: Utilize the `swap_test` method to assess the outcomes produced by the two quadratic functions. This quantum algorithm effectively compares quantum states, providing insight into the similarity or difference between the results of the quadratic evaluations.

In [ ]:
@QFunc
def main(comparison: Output[QBit]):
    #Coefficients for the quadratic functions
    a = 2
    b = 2

    #X Input
    x = QNum("x")

    #Y Outputs
    y = QNum("y")
    y2 = QNum("y2")


    allocate_num(2,False,0,x)
    hadamard_transform(x)

    inplace_quadratic(a,x,y) 
    inplace_quadratic(b,x,y2)

    #Compare two identical functions using Swap Test
    swap_test(y,y2,comparison)

Next, let's explore the quantum program created in the Classiq IDE. You can effortlessly visualize it by utilizing the `show()` function.

In [ ]:
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Now, let's examine the outcome. As depicted, we observe a singular result type, characterized by the comparison yielding a value of 0.0.

In [ ]:
def print_parsed_counts(job):
    results = job.result()
    parsed_counts = results[0].value.parsed_counts
    for parsed_state in parsed_counts: print(parsed_state.state)

job = execute(qprog)
print_parsed_counts(job)

Observe the overlap in the result states. As anticipated, due to the utilization of identical quadratic functions and inputs, our results are congruent. This demonstrates that the functions coincide across the entire domain of $[0,2^2)$.<br> In other words, all points of the function intersect.

In [ ]:
result = job.result()

state_overlap = np.sqrt(
    2*result[0].value.counts["0"] / sum(result[0].value.counts.values()) -1
)

print(f"States overlap from Swap-Test result: {round(state_overlap*100,3)}%")


Let's explore another scenario where the quadratic functions have slight variations. In this case, we'll compare $F_1(x) = 3 \cdot x^2$ and $F_2(x) = 2 \cdot x^2$.<br> We'll maintain the domain $[0, 2^2)$ for our comparison. Should you wish to extend the range further, it necessitates utilizing a backend capable of supporting a greater number of qubits.

In [ ]:
@QFunc
def main(comparison:Output[QBit]):
    a = 3
    b = 2
    x = QNum("x")
    y = QNum("y")
    y2 = QNum("y2")
    allocate_num(2,False,0,x)
    hadamard_transform(x)
    inplace_quadratic(a,x,y) 
    inplace_quadratic(b,x,y2)
    swap_test(y,y2,comparison)

In the implementation detailed above, we execute both $F_1(x)$ and $F_2(x)$, followed by a comparison of their outputs. Examining the state overlap from our Swap Test results, we anticipate less than 100% similarity, given not all inputs yield intersecting points. Nevertheless, due to the intrinsic similarity between these quadratic functions, a significant degree of overlap is expected.

In [ ]:
qmod = create_model(main)

qprog = synthesize(qmod)
show(qprog)
def print_parsed_counts(job):
    results = job.result()
    parsed_counts = results[0].value.parsed_counts
    for parsed_state in parsed_counts: print(parsed_state.state)

job = execute(qprog)
print_parsed_counts(job)

result = job.result()

state_overlap = np.sqrt(
    2 * result[0].value.counts["0"] / sum(result[0].value.counts.values()) - 1
)

print(f"States overlap from Swap-Test result: {round(state_overlap*100,3)}%")



The Swap Test, as demonstrated, operates at the qubit level to compare values. This powerful tool is adept at identifying quantum states that are either identical, completely non-overlapping, or partially overlapping. Its significance cannot be overstated, as it enables the comparison of quantum states without the risk of losing the quantum information contained within those states.

---

For our next exploration, we'll diverge by implementing two distinct quadratic functions, $F_1(x)$ and $F_2(x_2)$, utilizing two separate inputs, $x$ and $x_2$, each within the domain of $[0, 2^2)$. This approach allows us to investigate the behavior of non-identical quadratics under different input conditions."

In [ ]:
@QFunc
def main(comparison:Output[QBit]):
    a = 5
    b = 4
    x = QNum("x")
    x2 = QNum("x2")
    y = QNum("y")
    y2 = QNum("y2")
    allocate_num(2,False,0,x)
    
    allocate_num(2,False,0,x2)
    hadamard_transform(x2)
    inplace_quadratic(a,x,y) 
    inplace_quadratic(b,x2,y2)
    swap_test(y,y2,comparison)

In the previous example, we anticipate the overlap between the resulting quantum states to be under 50%. This is because the quantum states are less likely to be as similar as those in the previous example, where the functions and inputs were identical. It's crucial to understand that the Swap Test focuses on comparing the entirety of quantum states, rather than individual quantum digits or values. As such, the concept of state overlap might be less straightforward than observed in the initial example with identical functions and inputs.

In [ ]:
qmod = create_model(main)

qprog = synthesize(qmod)
show(qprog)
def print_parsed_counts(job):
    results = job.result()
    parsed_counts = results[0].value.parsed_counts
    for parsed_state in parsed_counts: print(parsed_state.state)

job = execute(qprog)
print_parsed_counts(job)

result = job.result()

state_overlap = np.sqrt(
    2 * result[0].value.counts["0"] / sum(result[0].value.counts.values()) - 1
)

print(f"States overlap from Swap-Test result: {round(state_overlap,3)}%")



---

## Conclusion

### The Swap Test: A Keystone in Quantum Computing

The journey through different scenarios vividly illustrates the Swap Test's crucial role in the quantum computing landscape. This method shines by enabling comprehensive comparisons of quantum states, revealing their similarities and differences without the risk of collapsing these states. Its significance transcends mere comparison, reflecting the quantum paradigm's unique approach to leveraging the intricate properties of quantum states. The Swap Test is essential for evaluating the performance of quantum algorithms, enhancing quantum simulations, and facilitating error correction, marking a significant stride towards realizing more complex quantum computing applications.

### The Classiq Platform: Empowering Quantum Innovation

The Classiq platform has emerged as an indispensable tool in the quantum computing industry. Its strength and importance lie in demystifying quantum programming, offering an accessible yet powerful avenue for users to design, prototype, and analyze quantum algorithms with ease. Classiq's user-friendly interface and advanced features bridge the gap between complex quantum theories and practical applications, enabling both beginners and seasoned experts to unlock quantum computing's vast potential. Through facilitating the design of sophisticated quantum functions, as demonstrated with our quadratic examples, Classiq accelerates the development of quantum applications, positioning itself as a catalyst for breakthroughs in computational power and innovation.

---

## Thank You!
#### Make sure to checkout the classiq platform here: https://platform.classiq.io